In [1]:
import glob
print(glob.glob("/share/*"))

['/share/input.txt', '/share/us-stock-dataset', '/share/Transformers1']


In [6]:
import sys
import os

import torch
from torch import nn as nn
import glob

from t4.transformer import FastTransformerConfig, FastTransformerRunner

from timeseries.csv_reader import read_and_merge_csv_files

from t3_karpathy.commons.commons import BaseTransformerConfig
from t3_karpathy.token_codec import TokenCodec
from t4.generic_dataloader import GenericDataloader

from t4.transformer import FastTransformerRunner



In [7]:
device = 'cuda'

torch.manual_seed(1337)

codec = TokenCodec()

config = FastTransformerConfig(codec.vocab_size, precision=torch.bfloat16, batch_size=32, block_size=512, n_embed=64, n_head=4, n_layer=4)

runner = FastTransformerRunner(config, codec.train_data, codec.val_data)

runner.train_iterate_n(2500)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(dataloader.token_codec.decode(runner.generate(context, max_new_tokens=2000)[0].tolist()))

File already downloaded
0.211146 M parameters
step 0: train loss 4.5173, val loss 4.5044, time/iter 2.384185791015625e-08
saved model
step 100: train loss 2.3372, val loss 2.3434, time/iter 0.017398669719696044
step 200: train loss 2.0557, val loss 2.0889, time/iter 0.015077710151672363
step 300: train loss 1.9266, val loss 1.9873, time/iter 0.01622471570968628
step 400: train loss 1.8332, val loss 1.9247, time/iter 0.016344649791717528
step 500: train loss 1.7719, val loss 1.8877, time/iter 0.015906867980957032
step 600: train loss 1.7274, val loss 1.8508, time/iter 0.013753623962402343
step 700: train loss 1.6897, val loss 1.8277, time/iter 0.01401778221130371
step 800: train loss 1.6598, val loss 1.8076, time/iter 0.01349508285522461
step 900: train loss 1.6379, val loss 1.7911, time/iter 0.015061068534851074
step 1000: train loss 1.6169, val loss 1.7834, time/iter 0.013109657764434814
saved model
step 1100: train loss 1.5987, val loss 1.7601, time/iter 0.013805181980133056
step 120

KeyboardInterrupt: 

In [ ]:
stocks_to_load = [
    "AAPL", "TSLA",
    "A", "GOOG", "AMZN", "PYPL", "NVDA", "AMD",
    "NFLX", "MSFT", "INTC", "CSCO", "ADBE", "CRM", "QCOM", "TXN", "AVGO",
    "INTU", "ORCL", "COST", "SBUX", "AMGN", "CHTR", "GILD", "CMCSA", "BKNG",
    "MDLZ", "FISV", "BIIB", "MU", "MCD", "AMAT", "ADP", "ILMN", "ATVI", "ISRG",
    "ADSK", "LRCX", "BIDU", "JD", "REGN", "WBA", "VRTX", "KHC", "WMT", "ZM", "MELI",
    "TMUS", "CTSH", "XLNX", "PCAR", "ALGN", "WDAY", "SIRI", "CTXS", "ADI", "EXC", "LULU",
    "MAR", "KLAC", "PAYX", "EA", "ILMN", "ALXN", "MNST", "BMRN", "EBAY", "CTAS", "VRSK",
    "IDXX", "CDNS", "NXPI", "ASML", "INCY", "KLAC", "MCHP", "SNPS", "SWKS", "VRSN",
    "WDC", "WYNN", "XLNX", "ZBRA", "ZTS", "AEP", "AIG", "ALL", "AXP", "BA", "BAC",
    "BK", "BLK", "C", "CAT", "CL", "COF", "COP", "COST", "CSCO", "CVS", "CVX",
    "DD", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD",
    "GM", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI",
    "KO", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM",
    'BLND', 'BLNG', 'BLNKW', 'BLNGU', 'BLNGW', 'BLNK', 'BLPH', 'BLRX',
    'BLTE', 'BLU', 'BLUA', 'BLUE', 'BLW', 'BLX', 'BLZE', 'BMA', 'BMAC', 'BMAQ', 'BMAQR', 'BMAQU'
]

directory_path = '/share/us-stock-dataset/Data/Stocks'

# stocks_to_load = [s.split("\\")[1].replace(".csv", "") for s in glob.glob(directory_path + "//*.csv")]
#
# stock_number_to_load = 200
#
# stocks_to_load = stocks_to_load[1000:1000+stock_number_to_load]

dataloader = TimeseriesDataloader(directory_path, stocks_to_load)
config = TimeseriesTransformerConfig(
    batch_size=32,
    block_size=512,
    n_embed=32,
    n_head=4,
    n_layer=8,
    kernel_size=1,
    learning_rate=1e-3,
    channels=dataloader.get_number_of_channels()
)
model = TimeseriesTransformerModel(config)
trainer1 = TimeseriesPandasTrainer(config, dataloader=dataloader, model=model)

trainer1.train_eval(20000)
